In [3]:
from sklearn.datasets import fetch_covtype
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, Input, Model

In [7]:
data = fetch_covtype()
X = data.data
y = data.target - 1

In [8]:
X

array([[2.596e+03, 5.100e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.590e+03, 5.600e+01, 2.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.804e+03, 1.390e+02, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.386e+03, 1.590e+02, 1.700e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.384e+03, 1.700e+02, 1.500e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.383e+03, 1.650e+02, 1.300e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [9]:
y

array([4, 4, 1, ..., 2, 2, 2], dtype=int32)

In [10]:
data

{'data': array([[2.596e+03, 5.100e+01, 3.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.590e+03, 5.600e+01, 2.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.804e+03, 1.390e+02, 9.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [2.386e+03, 1.590e+02, 1.700e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.384e+03, 1.700e+02, 1.500e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [2.383e+03, 1.650e+02, 1.300e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00]]),
 'target': array([5, 5, 2, ..., 3, 3, 3], dtype=int32),
 'frame': None,
 'target_names': ['Cover_Type'],
 'feature_names': ['Elevation',
  'Aspect',
  'Slope',
  'Horizontal_Distance_To_Hydrology',
  'Vertical_Distance_To_Hydrology',
  'Horizontal_Distance_To_Roadways',
  'Hillshade_9am',
  'Hillshade_Noon',
  'Hillshade_3pm',
  'Horizontal_Distance_To_Fire_Points',
  'Wilderness_Area_0',
  'Wilderness_Area_1',
  'Wilderness_Area_2',

In [11]:
X_num = X[:, :10] # numerical features
X_cat = X[:, 10:] #catgeorical features

In [12]:
scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num) # normalise the data

In [13]:
X_scaled = np.concatenate([X_num_scaled, X_cat], axis=1)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42) # 80-20 split

In [15]:
X_batch = X_train[:128]
y_batch = y_train[:128] # batch for overfitting

In [16]:
def build_multi_class_residual_mlp(input_dim, num_classes):
    inputs = Input(shape=(input_dim,), name='Input')

    # Initial Dense
    x = layers.Dense(32, activation='relu', name='Initial_Dense')(inputs)

    # Residual Block
    res_input = x
    res = layers.Dense(32, activation='relu', name='Res1')(res_input)
    res = layers.Dense(32, name='Res2')(res)
    res = layers.Add(name='Residual_Add')([res_input, res])
    res = layers.ReLU(name='Residual_ReLU')(res)

    # Additional Skip Connection (from input to later)
    skip = layers.Dense(32, activation='relu', name='Skip_Connection')(inputs)

    # Combine residual + skip
    combined = layers.Add(name='Combine_Skip_Residual')([res, skip])

    # Final Dense layers
    final = layers.Dense(16, activation='relu', name='Final_Dense')(combined)
    outputs = layers.Dense(num_classes, activation='softmax', name='Output')(final)

    model = Model(inputs=inputs, outputs=outputs, name='Residual_MLP_MultiClass')
    return model

model = build_multi_class_residual_mlp(input_dim=X_scaled.shape[1], num_classes=7)
model.summary()

Model: "Residual_MLP_MultiClass"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)        │ (None, 54)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Initial_Dense (Dense)     │ (None, 32)             │          1,760 │ Input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Res1 (Dense)              │ (None, 32)             │          1,056 │ Initial_Dense[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Res2 (Dense)              │ (None, 32)             │          1,056 │ Res1[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Residual_Add (Add)        │ (None, 32)             │              0 │ Initial_Dense[0][0],   │
│                           │                        │                │ Res2[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Residual_ReLU (ReLU)      │ (None, 32)             │              0 │ Residual_Add[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Skip_Connection (Dense)   │ (None, 32)             │          1,760 │ Input[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Combine_Skip_Residual     │ (None, 32)             │              0 │ Residual_ReLU[0][0],   │
│ (Add)                     │                        │                │ Skip_Connection[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Final_Dense (Dense)       │ (None, 16)             │            528 │ Combine_Skip_Residual… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Output (Dense)            │ (None, 7)              │            119 │ Final_Dense[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 6,279 (24.53 KB)

 Trainable params: 6,279 (24.53 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train to overfit
history = model.fit(X_batch, y_batch,
                    epochs=20,
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.4844 - loss: 1.4137 - val_accuracy: 0.4675 - val_loss: 1.4615
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5302 - loss: 1.2924 - val_accuracy: 0.4692 - val_loss: 1.3995
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5458 - loss: 1.2166 - val_accuracy: 0.4764 - val_loss: 1.3513
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5417 - loss: 1.1842 - val_accuracy: 0.4814 - val_loss: 1.3139
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5031 - loss: 1.1652 - val_accuracy: 0.4887 - val_loss: 1.2851
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5219 - loss: 1.1631 - val_accuracy: 0.4968 - val_loss: 1.2615
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5802 - loss: 1.0925 - val_accuracy: 0.5036 - val_loss: 1.2410
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5823 - loss: 0.9672 - val_accuracy: 0.5098 - val_loss: 1.2235
Epoch 9/20
4/4

In [19]:
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)

print(f"\n--- Final Results ---")
print(f"Number of parameters: {model.count_params()}")
print(f"Final training loss: {history.history['loss'][-1]:.6f}")
print(f"Final validation loss: {val_loss:.6f}")



--- Final Results ---
Number of parameters: 6279
Final training loss: 0.761687
Final validation loss: 1.082370


In [22]:
model.save("residual_multiclass_model.h5")

In [24]:
from google.colab import files
files.download("residual_multiclass_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>